# Cats vs Dogs

## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

In [ ]:
# Permet d'éviter les erreurs mémoires pour le GPU
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
def plot_scores(train) :
    accuracy = train.history['acc']
    val_accuracy = train.history['val_acc']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
# Affichage des fichiers dans le répertoire de données Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Importation des images

On utilise *ImageDataGenerator*, à la fois pour charger les images à partir d'un répertoire, et pour augmenter le nombre d'images.

On augmente le nombre d'images en appliquant des transformations simples (rotation, retournement, décalage, ...)
- **rotation_range** est une valeur en degrés (0-180), une plage à l'intérieur de laquelle les images peuvent tourner aléatoirement
- **width_shift** et **height_shift** sont des plages (en tant que fraction de la largeur ou de la hauteur totale) dans lesquelles les images peuvent être traduites aléatoirement verticalement ou horizontalement.
- **rescale** est une valeur par laquelle nous allons multiplier les données avant tout autre traitement. Nos images originales consistent en des coefficients RVB dans le 0-255, mais ces valeurs seraient trop élevées pour que nos modèles puissent les traiter (compte tenu d'un taux d'apprentissage typique), alors nous ciblons des valeurs entre 0 et 1 en utilisant plutôt un facteur 1/255.
- **shear_range** est pour l'application aléatoire de transformations de cisaillement
- **zoom_range** est pour zoomer aléatoirement à l'intérieur des images
- **horizontal_flip** est pour retourner aléatoirement la moitié des images horizontalement --pertinent quand il n'y a pas de suppositions d'asymétrie horizontale (par exemple, des images du monde réel).
- **fill_mode** est la stratégie utilisée pour remplir les pixels nouvellement créés, qui peuvent apparaître après une rotation ou un décalage largeur/hauteur.



In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

valid_datagen = ImageDataGenerator(
        rescale=1./255)

*flow_from_directory* permet de lire les images directement à partir d'un répertoire.  
Ici les images d'apprentissage sont dans un répertoire **CatsDogs2\train*

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory="../input/cat-and-dog/training_set/training_set/",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory="../input/cat-and-dog/test_set/test_set/",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=1,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
print(train_generator.n)

In [ ]:
plt.imshow(*valid_generator[0][0])

In [ ]:
print(valid_generator[0][1])

## Modèle CNN

On teste un modèle avec deux couches convolutionnelles :

In [ ]:
# Modèle CNN plus profond
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Le nombres d'images générées à chaque epoch est *steps_by_epoch x batch_size*. Ici on génère 10 fois plus d'images à partir du dataset d'apprentissage (ce qui ralentit évidemment l'apprentissage)

In [ ]:
STEP_SIZE_TRAIN=3*train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [ ]:
model.evaluate_generator(generator=valid_generator)

In [ ]:
# Prediction
y_pred = model.predict_generator(valid_generator,verbose=1).argmax(axis=-1)

In [ ]:
import random

labels = ['cat','dog']
plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
for i in range(1,50) :
    ir = random.randint(0,n_test)
    plt.subplot(10,5,i)
    plt.axis('off')
    plt.imshow(*valid_generator[ir][0])
    plt.title(labels[y_pred[ir]])
#    plt.title('%s / %s' % (y_cnn_classe[ir], y_test_classe[ir]))

## Importation des images

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_datagen = ImageDataGenerator(
        rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory="CatsDogs2\\train",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory="CatsDogs2\\valid",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [ ]:
model.evaluate_generator(generator=valid_generator)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory="CatsDogs2\\train",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory="CatsDogs2\\valid",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
# Modèle VGG16

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(64, 64, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [ ]:
model.evaluate_generator(generator=valid_generator)

In [ ]:
from keras.applications import VGG16
model = Sequential()
model.add(VGG16(weights=None, input_tensor=None, input_shape=(64,64,3), classes=2))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

In [ ]:
from keras.applications import ResNet
model = Sequential()
model.add(ResNet(weights=None, input_tensor=None, input_shape=(64,64,3), classes=2))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)